<a href="https://colab.research.google.com/github/josejailson/hugging_face/blob/main/hugging_face_model_on_sagemaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the model

Hugging Face is a community and data science platform wich provides tools tha unable users to build, train and deploy ML models based on open source code and technologies.

The model we'll use here is *distilbert-base-uncased-finetuned-sst-2-english*, an English text classification based on the DistilBERT.
By the way, DistilBERT is a transformer model, smaller and faster than BERT. It was pretrained on the same corpus and used the BERT base model as a teacher.
For more information, this model is hosted at https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english.

So, let's introduce the model.

In [1]:
%pip install huggingface_hub --quiet
%pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TextClassificationPipeline


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

Here is the sentiment analysis of the model on the phrase "I love Amazon SageMaker Studio Lab!":

In [ ]:
pipe("A love Amazon SageMaker Studio Lab!")

# Set up the pre-requisitions

For start, we must install AWS CLI, boto3 and configure the AWS credentials.

In [4]:
%pip install boto3 --quiet

In [5]:
%pip install awscli  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 18.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
%mkdir ~/.aws

In [7]:
%%writefile ~/.aws/credentials

[default]
aws_access_key_id =
aws_secret_access_key =

Writing /root/.aws/credentials


In [8]:
%%writefile ~/.aws/config

[default]
region=us-east-1

Writing /root/.aws/config


A additional step is create a SageMake IAM Execution Role, in order to grant permissions.

In [27]:
role = "arn:aws:iam::875652382423:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole"

# Connecting to AWS using boto3

Since our credentials are in the source .aws, connect to AWS is straightforward.

In [11]:
import boto3

sm_client = boto3.client("sagemaker")
response = sm_client.list_endpoints()

len(response)

2

For traking actions on AWS, it's a good ideia name the model and the endpoint with the data when they were created.

In [12]:
import time

ml_model_name = "text-classification-hugging-face"
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name = ml_model_name + '-model' + timestamp
endpoint_config_name = ml_model_name + "-epc" + timestamp
print(model_name)
print(endpoint_config_name)

text-classification-hugging-face-model-2023-07-07-15-53-02
text-classification-hugging-face-epc-2023-07-07-15-53-02


# Creating a model with SageMaker Client

The core component of this code is the container_config dictionary, where all model is created.

In [28]:
model_data_url = "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.9-transformers4.12-cpu-py38-ubuntu20.04"

container_config = {"Image": model_data_url,
                    "Mode": "SingleModel",
                    "Environment": {
                        "HF_MODEL_ID": "distilbert-base-uncased-finetuned-sst-2-english",
                        "HF_TASK": "text-classification",
                        "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
                        "SAGEMAKER_REGION": "us-east-1"
                    }
                    }

response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer=container_config,
    ExecutionRoleArn=role,
    EnableNetworkIsolation=False
)

# Creating a serveless endpoint configuration

In [33]:
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "AllTraffic",
            "ServerlessConfig": {
                "MemorySizeInMB": 3072,
                "MaxConcurrency": 10
            }
        }
    ]
)



ClientError: ignored

In [36]:
endpoint_name = "studio-lab-ep" + '-epc' + timestamp


In [37]:
endpoint_name

'studio-lab-ep-epc-2023-07-07-15-53-02'

In [ ]:
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

# Testing the model

In [35]:
import json
import boto3

runtime = boto3.client("sagemaker-runtime")

content_type = "application/json"

data = {
    "inputs": "Happy Birthday to you!"
}

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=json.dumps(data)
)

ValidationError: ignored

In [ ]:
response["Body"].read().decode()

# Deleting the model, the endpoint confi and the endpoint

In [ ]:
sm_client.delete_endpoint()
sm_client.delete_endpoint_config()
sm_client.delete_model()